# This notebook was implemented on Colab

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip install keras-tuner

In [ ]:
# Standard library imports
import json
import os
import random
import shutil

# Third-party imports
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_tuner.tuners import RandomSearch
from kerastuner import HyperModel, Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import BatchNormalization

# Local application/library specific imports
from google.colab import drive

<ipython-input-2-fd9ef17bd841>:13: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel, Hyperband


In [ ]:
# Check if TPU is available
if 'COLAB_TPU_ADDR' in os.environ:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  tpu_strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print("Running on TPU.")
else:
  tpu_strategy = None
  print("Not running on TPU.")


Not running on TPU.


In [ ]:
# mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Images Classification and splitting ... train 80% - validate 20%

In [ ]:
import os
import shutil
import random

# Define the paths for the input and output directories
input_dir = "output"
output_dir = 'data_categorized'

# Create the output directories if they do not exist
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

train_dir = os.path.join(output_dir, "train")
if not os.path.exists(train_dir):
    os.mkdir(train_dir)

valid_dir = os.path.join(output_dir, "test")
if not os.path.exists(valid_dir):
    os.mkdir(valid_dir)

# Define a dictionary to hold the counts of each image type
counts = {}

# Loop through each file in the input directory
for filename in os.listdir(input_dir):
    # Extract the image type from the filename
    img_type = filename.split("_")[0]

    # Increment the count of the current image type in the dictionary
    counts[img_type] = counts.get(img_type, 0) + 1

    # Determine whether the current image should be included in the training set or the validation set
    if random.random() < 0.8:
        output_subdir = train_dir
    else:
        output_subdir = valid_dir

    # Create the output directory for the current image type if it does not exist
    type_dir = os.path.join(output_subdir, img_type)
    if not os.path.exists(type_dir):
        os.mkdir(type_dir)

    # Copy the image file to the appropriate output directory
    src_path = os.path.join(input_dir, filename)
    dst_path = os.path.join(type_dir, filename)
    shutil.copy(src_path, dst_path)

# # Print the counts of each image type in the dictionary
# print(counts)

In [ ]:
!unrar x '/content/drive/MyDrive/AML/data_categorized.rar' '/content/drive/MyDrive/AML/'

### Data Preprocessing

In [ ]:
train_directory = '/content/drive/MyDrive/AML/data_categorized/train'
test_directory = '/content/drive/MyDrive/AML/data_categorized/test'

In [ ]:
# Set up ImageDataGenerator objects to perform preprocessing and data augmentation
from keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(
    rescale=1./255,       # Normalize pixel values between 0 and 1
    shear_range=0.4,      # Randomly apply shearing transformations
    zoom_range=0.3,       # Randomly zoom in on images
    horizontal_flip=True  # Randomly flip images horizontally
    )

validaiton_data_generator = ImageDataGenerator(rescale=1./255)
batch_size = 32
target_size = (200, 200), # Resize images to 200x200

# prepare an iterators for each dataset .... 2 iterators will be created

# Set up training data generator to read images from directory and preprocess them
train_generator = train_data_generator.flow_from_directory(train_directory,
                                                    target_size = target_size,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical', # multi-class classification
                                                    )

# Set up validaiton data generators to read images from directories and preprocess them 
validation_generator = validaiton_data_generator.flow_from_directory(test_directory,
                                                target_size = target_size,
                                                batch_size = batch_size,
                                                class_mode = 'categorical' # for multi-class classification
                                                )

Found 71922 images belonging to 9 classes.
Found 18078 images belonging to 9 classes.


## Modeling

In [ ]:
def build_model(hp):
    """ build the model with different values of hyperparameters to tune

    Args:
        hp: hyperparameter

    Returns:
        model
    """
    num_classes = len(train_generator.class_indices)
    model = keras.Sequential()
    model.add(layers.Input(shape=(200, 200, 3))) # width, height, rgb
    
    # Tune the number of hidden dense layers between 1-3
    for i in range(hp.Int('num_layers', 1, 3)):
        
        # Tune number of neurons for each layer
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                               activation=hp.Choice('activation_' + str(i), values=['relu', 'sigmoid', 'tanh'])
                               )
        )
        
        # Add Dropout layer if the use_dropout hyperparameter is True
        if hp.Boolean('use_dropout_' + str(i)):
          model.add(layers.Dropout(rate=hp.Float('dropout_rate_' + str(i), min_value=0.0, max_value=0.5, step=0.3)))  # model.add(layers.Dropout(rate=0.2))

        model.add(BatchNormalization())
        
    # Tune the regularization rate for the kernel weights
    # Choose an optimal value between 0.01, 0.001, and 0.0001
    hp_kernel_regularizer = hp.Choice('kernel_regularizer_' + str(i), [0.01, 0.001, 0.0001])
    model.add(layers.Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(hp_kernel_regularizer)))
    

    model.add(layers.Flatten())

    # Add the output layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    # Compile the model with the selected hyperparameters
    # Tune the learning rate
    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')), 
        loss=hp.Choice('loss_function', values=['sparse_categorical_crossentropy', 'categorical_crossentropy']), # loss='categorical_crossentropy', 
        metrics=['accuracy'])
    
    return model

### Hyperprameter tuning

In [ ]:
# Define the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    # max_trials=5,
    # executions_per_trial=4,
    overwrite=True,
    directory='/content/drive/MyDrive/AML/final_model',
    project_name='final_object_classification')

# tuner = Hyperband(
#     build_model,
#     objective='val_accuracy',
#     # max_epochs=10,
#     # factor=3,
#     directory='/content/drive/MyDrive/AML/final_model',
#     project_name='final_object_classification')

In [ ]:
# Search for the best hyperparameters
tuner.search(train_generator,
             validation_data=validation_generator,
             epochs=3,
             verbose=2)

Trial 3 Complete [00h 00m 13s]

Best val_accuracy So Far: 0.1131209209561348
Total elapsed time: 01h 03m 29s


In [ ]:
tuner

In [ ]:
tuner.get_best_models(num_models=1)

In [ ]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model

In [ ]:
tuner.get_best_models()

In [ ]:
# Print the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best number of layers: ", best_hyperparameters.get('num_layers'))
for i in range(best_hyperparameters.get('num_layers')):
    print("Best number of units in layer " + str(i) + ": ", best_hyperparameters.get('units_' + str(i)))
    print("Best activation function in layer " + str(i) + ": ", best_hyperparameters.get('activation_' + str(i)))
print("Best learning rate: ", best_hyperparameters.get('learning_rate'))

Best number of layers:  1
Best number of units in layer 0:  480
Best activation function in layer 0:  tanh
Best learning rate:  0.0009313969589226206


In [ ]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model with the best hyperparameters
model_best_hp__tuner_BatchNormalization_dropout_regularization = build_model(best_hps)

# Print the best hyperparameters
print("Best Hyperparameters:")
print(f"num_layers: {best_hps.get('num_layers')}")

# if best num_layers > 1
for i in range(best_hps.get('num_layers')):
    print(f"units_{i}: {best_hps.get('units_' + str(i))}")
    print(f"activation_{i}: {best_hps.get('activation_' + str(i))}")
    print(f"use_dropout_{i}: {best_hps.get('use_dropout_' + str(i))}")
    if best_hps.get('use_dropout_' + str(i)):
        print(f"dropout_rate_{i}: {best_hps.get('dropout_rate_' + str(i))}")

print(f"kernel_regularizer_{i}: {best_hps.get('kernel_regularizer_' + str(i))}")
print(f"learning_rate: {best_hps.get('learning_rate')}")
print(f"loss_function: {best_hps.get('loss_function')}")

Best Hyperparameters:
num_layers: 1
units_0: 480
activation_0: tanh
use_dropout_0: False
kernel_regularizer_0: 0.01
learning_rate: 0.0009313969589226206
loss_function: categorical_crossentropy


In [ ]:
# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]

# Print the name and value of each hyperparameter
for hp_name, hp_value in best_hp.values.items():
    print(f'{hp_name}: {hp_value}')

num_layers: 1
units_0: 480
activation_0: tanh
use_dropout_0: False
kernel_regularizer_0: 0.01
learning_rate: 0.0009313969589226206
loss_function: categorical_crossentropy


In [ ]:
# Save the best hyperparameters
best_hps_dict = best_hps.values
with open('/content/drive/MyDrive/AML/best_hps_dict__tuner_BatchNormalization_dropout_regularization.json', 'w') as f:
    json.dump(best_hps_dict, f)

# Save the best model
model_best_hp__tuner_BatchNormalization_dropout_regularization.save('/content/drive/MyDrive/AML/best_model__tuner_BatchNormalization_dropout_regularization.h5')

## Overfitting removal approach:
using early stopping technique

In [ ]:
# Define early stopping callback
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model with early stopping
history = best_model.fit(train_generator,
                         validation_data=validation_generator, # , steps_per_epoch=16
                         epochs=3, callbacks=[early_stopping] # callbacks=[EarlyStopping(patience=5)]
                         )

Epoch 1/3
2248/2248 [==============================] - 1316s 585ms/step - loss: 13.1739 - accuracy: 0.1087 - val_loss: 13.0953 - val_accuracy: 0.1090
Epoch 2/3
2248/2248 [==============================] - 1269s 564ms/step - loss: 14.0787 - accuracy: 0.1106 - val_loss: 19.5486 - val_accuracy: 0.1090
Epoch 3/3
2248/2248 [==============================] - 1292s 575ms/step - loss: 13.8709 - accuracy: 0.1124 - val_loss: 14.0395 - val_accuracy: 0.1110


In [ ]:
history.history

{'loss': [13.173863410949707, 14.078662872314453, 13.870908737182617],
 'accuracy': [0.10865937918424606, 0.11055032163858414, 0.11242735385894775],
 'val_loss': [13.095309257507324, 19.54863739013672, 14.039475440979004],
 'val_accuracy': [0.1089722290635109, 0.1089722290635109, 0.1109636053442955]}

In [ ]:
# Access the best model
early_stopping.model.history.history

{'loss': [13.173863410949707, 14.078662872314453, 13.870908737182617],
 'accuracy': [0.10865937918424606, 0.11055032163858414, 0.11242735385894775],
 'val_loss': [13.095309257507324, 19.54863739013672, 14.039475440979004],
 'val_accuracy': [0.1089722290635109, 0.1089722290635109, 0.1109636053442955]}

In [ ]:
# # Save the history object
# with open('/content/drive/MyDrive/AML/history_all_hp_early_stopping.pkl', 'wb') as f:
#     pickle.dump(history.history, f)